### 얼굴 필터 적용

In [ ]:
import cv2
import mediapipe as mp
import numpy as np

# 람쥐 필터 이미지 (투명 PNG)
filter_img = cv2.imread('squirrel1.png', cv2.IMREAD_UNCHANGED)

# mediapipe 얼굴 검출 초기화
mp_face = mp.solutions.face_detection
mp_draw = mp.solutions.drawing_utils
face_detection = mp_face.FaceDetection(model_selection=0, min_detection_confidence=0.5)

# 필터 오버레이 함수
def overlay_filter(frame, filter_img, x, y, w, h):
    filter_resized = cv2.resize(filter_img, (w, h))

    for c in range(3):  # BGR 채널
        frame[y:y+h, x:x+w, c] = frame[y:y+h, x:x+w, c] * (1 - filter_resized[:, :, 3]/255.0) + \
                                 filter_resized[:, :, c] * (filter_resized[:, :, 3]/255.0)

# 웹캠 열기
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 좌우 반전
    frame = cv2.flip(frame, 1)
    h, w, _ = frame.shape

    # 얼굴 감지
    results = face_detection.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    if results.detections:
        for detection in results.detections:
            bboxC = detection.location_data.relative_bounding_box
            x = int(bboxC.xmin * w)
            y = int(bboxC.ymin * h)
            box_w = int(bboxC.width * w)
            box_h = int(bboxC.height * h)

            # 얼굴 기준으로 필터 위치 조정 (이마 쪽으로 위로 이동)
            offset_y = int(box_h * 0.4)
            x1 = max(0, x - int(box_w * 0.2))
            y1 = max(0, y - offset_y)
            x2 = min(w, x1 + int(box_w * 1.4))
            y2 = min(h, y1 + int(box_h * 1.5))

            filter_w = x2 - x1
            filter_h = y2 - y1

            if filter_w > 0 and filter_h > 0:
                overlay_filter(frame, filter_img, x1, y1, filter_w, filter_h)

    cv2.imshow('Squirrel Filter 🐿️', frame)

    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 

#### 눈감으면 다른 가면

In [1]:
import cv2
import mediapipe as mp
import numpy as np

# 두 이미지 불러오기
filter_eye_closed = cv2.imread('squirrel1.png', cv2.IMREAD_UNCHANGED)
filter_eye_open = cv2.imread('squirrel2.png', cv2.IMREAD_UNCHANGED)

# mediapipe 초기화
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.5)

# 눈 감김 판단 함수 (EAR 계산)
def get_ear(landmarks, indices, img_w, img_h):
    pts = [(int(landmarks[i].x * img_w), int(landmarks[i].y * img_h)) for i in indices]
    A = np.linalg.norm(np.array(pts[1]) - np.array(pts[5]))
    B = np.linalg.norm(np.array(pts[2]) - np.array(pts[4]))
    C = np.linalg.norm(np.array(pts[0]) - np.array(pts[3]))
    ear = (A + B) / (2.0 * C)
    return ear

# 필터 오버레이 함수
def overlay_filter(frame, filter_img, x, y, w, h):
    filter_resized = cv2.resize(filter_img, (w, h))
    for c in range(3):
        frame[y:y+h, x:x+w, c] = frame[y:y+h, x:x+w, c] * (1 - filter_resized[:, :, 3]/255.0) + \
                                 filter_resized[:, :, c] * (filter_resized[:, :, 3]/255.0)

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.flip(frame, 1)
    h, w, _ = frame.shape
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # 눈 감김 판단용 좌표
            left_eye_indices = [33, 160, 158, 133, 153, 144]
            right_eye_indices = [362, 385, 387, 263, 373, 380]
            left_ear = get_ear(face_landmarks.landmark, left_eye_indices, w, h)
            right_ear = get_ear(face_landmarks.landmark, right_eye_indices, w, h)
            avg_ear = (left_ear + right_ear) / 2.0

            # 얼굴 전체 박스 좌표 구하기
            xs = [lm.x for lm in face_landmarks.landmark]
            ys = [lm.y for lm in face_landmarks.landmark]
            min_x = int(min(xs) * w)
            max_x = int(max(xs) * w)
            min_y = int(min(ys) * h)
            max_y = int(max(ys) * h)

            face_w = max_x - min_x
            face_h = max_y - min_y
            cx = min_x - int(face_w * 0.1)
            cy = min_y - int(face_h * 0.4)
            cw = int(face_w * 1.2)
            ch = int(face_h * 1.5)

            # 눈 감았는지에 따라 필터 선택
            filter_to_use = filter_eye_closed if avg_ear < 0.2 else filter_eye_open

            # 오버레이
            cx = max(0, min(cx, w - cw))
            cy = max(0, min(cy, h - ch))
            overlay_filter(frame, filter_to_use, cx, cy, cw, ch)

    cv2.imshow("Squirrel Filter 🐿️", frame)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


c:\Users\SSAFY\AppData\Local\Programs\Python\Python310\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


KeyboardInterrupt: 